In [28]:
import os, logging

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow_hub").setLevel(logging.CRITICAL)

import tensorflow as tf
import tensorflow_hub as hub

In [29]:
#import tensorflow as tf
import environment
import math
import numpy as np
import random
from collections import deque
import matplotlib.pyplot as plt

In [31]:
env = environment

n_episodes = 1000
n_win_ticks = 1000

gamma = 1.
epsilon = 1.
epsilon_min = 0.01
epsilon_decay = 0.999
alpha = 0.001  
alpha_decay = 0.001

batch_size = 1000

memory = deque(maxlen = 1000)
scores = deque(maxlen = 100)

SHOW_EVERY = 1
PLOT_EVERY = 100
show =  True

In [32]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(Dense(24, input_dim = 11, activation = 'relu'))
model.add(Dense(48, activation = 'relu'))
model.add(Dense(48, activation = 'relu'))
model.add(Dense(3, activation = 'relu'))
model.compile(loss = 'mse', optimizer = Adam(lr = alpha, decay = alpha_decay))

In [33]:
def save(state, action, reward, next_state, done):
    memory.append((state, action, reward, next_state, done))   
    
def choose_action(state, epsilon):
    if np.random.random() <= epsilon:
        return env.sample() 
    else: 
        return np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon, 1.0 - math.log10((t+1)*epsilon_decay)))

def replay(batch_size, epsilon):
    
    x_batch, y_batch = [], []
    minibatch = random.sample(memory, min(len(memory), batch_size))
    
    for state, action, reward, next_state, done in minibatch:  
        
        y_target = model.predict(state)
        y_target[0][action] = -100 if done else reward + gamma*np.max(model.predict(next_state)[0])
        for act in range(3):
            if y_target[0][act] < 10:
                y_target[0][act] = 10
        x_batch.append(state[0])
        y_batch.append(y_target[0])
    
    model.fit(np.array(x_batch), np.array(y_batch), batch_size = len(x_batch), verbose = 0)
    
    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

In [9]:
model = tf.keras.models.load_model('model1.h5')
epsilon = 0

In [34]:
def run():
    ep = []
    ticks = []
    mean_score = []
    # make the environment
    env = environment
    env.reset()
    env.show = show
    # run the loop for n_episodes
    for e in range(n_episodes):
        # Reset the environment
        env.reset()
        done = False
        # First state
        state = env.get_state()
        # Reset the score counter
        i = 0
        # Run the loop - algorithm
        while not done:
            # get the action according the state.
            action = choose_action(state, epsilon)
            # Make the move
            env.step(action)
            # Get the next state from environment
            next_state = env.get_state()
            # Get the reward
            reward = env.get_reward()
            # Check if the programm has done
            done = env.done
            # Save the current situation
            save(state, action, reward, next_state, done)
            # Update the state
            state = next_state
            # Raise up the score
            i += 1
        
        print("Episode {} run for {} ticks".format(e, i))
        replay(batch_size, get_epsilon(e))
        
        ep.append(e)
        ticks.append(i)
        scores.append(i)
        mean_score.append(np.mean(scores))
        
        plt.plot(ep, ticks, 'b')
        plt.plot(ep, mean_score, 'r')
        plt.xlabel('Episode')
        plt.ylabel('Ticks')
        if e % PLOT_EVERY == 0 and e > 1:
            plt.show()
        
        if show == True:
            if e % SHOW_EVERY == 0:
                env.show = True
            else:
                env.show = False

In [ ]:
run()

Episode 0 run for 92 ticks


Exception ignored in: <function ScopedTFGraph.__del__ at 0x0000020DA8135168>
Traceback (most recent call last):
  File "C:\Users\EGiannakidis\Anaconda3\envs\py37\lib\site-packages\tensorflow_core\python\framework\c_api_util.py", line 52, in __del__
    c_api.TF_DeleteGraph(self.graph)
AttributeError: 'ScopedTFGraph' object has no attribute 'graph'


Episode 1 run for 120 ticks
Episode 2 run for 67 ticks
Episode 3 run for 55 ticks
Episode 4 run for 59 ticks
Episode 5 run for 78 ticks
Episode 6 run for 85 ticks
Episode 7 run for 98 ticks
Episode 8 run for 68 ticks
Episode 9 run for 50 ticks
Episode 10 run for 75 ticks
Episode 11 run for 87 ticks
Episode 12 run for 54 ticks
Episode 13 run for 87 ticks
Episode 14 run for 64 ticks
Episode 15 run for 84 ticks
Episode 16 run for 78 ticks
Episode 17 run for 60 ticks
Episode 18 run for 81 ticks
Episode 19 run for 95 ticks
Episode 20 run for 95 ticks
Episode 21 run for 104 ticks
Episode 22 run for 104 ticks
Episode 23 run for 62 ticks
Episode 24 run for 79 ticks
Episode 25 run for 59 ticks
Episode 26 run for 48 ticks
Episode 27 run for 63 ticks
Episode 28 run for 59 ticks
Episode 29 run for 51 ticks
Episode 30 run for 83 ticks
Episode 31 run for 88 ticks


In [8]:
model.save('model1.h5')

In [5]:
new_model = tf.keras.models.load_model('model1.h5')

In [7]:
#model = tensorflow.keras.models.load_model('model1.h5')
def test(model):
    i = 0
    for _ in range(10):
        i = 0
        env = environment
        env.show = True
        env.reset()
        done = False
        while not done:
            state = env.get_state()
            action = np.argmax(model.predict(state))
            env.step(action)
            done = env.done
            i += 1
        print("The total ticks was {}".format(i))

In [8]:
test(new_model)

The total ticks was 91
The total ticks was 100
The total ticks was 100
The total ticks was 90
The total ticks was 90
The total ticks was 91
The total ticks was 100
The total ticks was 100


error: display Surface quit